# Bibliotecas

In [ ]:
! pip install selenium --quiet
! apt-get update --quiet
! apt install chromium-chromedriver --quiet

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:6 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:7 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists...
Reading package lists...
Building dependency tree...
Reading state information...
chromium-chromedriver is already the newest version (1:85.0.4183.83-0ubuntu2.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 16 not upgraded.


In [ ]:
from    selenium import webdriver
import  pandas as pd
from    selenium import webdriver
import  numpy as np
import  re
import  plotly.express as px

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Webdriver

In [ ]:
options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')

# Nikola Jokic

In [ ]:
# wd = webdriver.Chrome(options=options)

# temporadas_jokic = np.arange(2016, 2024)

# lista_dfs = []

# # Coleta os dados do jogador de 2016 a 2023
# for ano in temporadas_jokic:
#   site = 'https://www.espn.com.br/nba/jogador/jogo-a-jogo/_/id/3112335/tipo/nba/ano/' + str(ano)

#   wd.get(site)

#   html = wd.page_source

#   scrapping = pd.read_html(html)

#   df_stats_ano = pd.concat([scrapping[0],
#                             scrapping[1],
#                             scrapping[2],
#                             scrapping[3],
#                             scrapping[4],
#                             scrapping[5],
#                             scrapping[6]])

#   df_stats_ano.insert(0, 'Temporada', ano)

#   lista_dfs.append(df_stats_ano)

#   df_stats_jokic = pd.concat(lista_dfs).reset_index(drop=True)

In [ ]:
# df_stats_jokic.to_csv('/content/drive/MyDrive/NBA Analysis/nba_stats_jokic_raw.csv',
#                       encoding='utf-8',
#                       sep=',',
#                       index=False)

# Dataset raw from csv

In [ ]:
df_stats_jokic = pd.read_csv('/content/drive/MyDrive/NBA Analysis/nba_stats_jokic_raw.csv',
                             encoding='utf-8',
                             sep=',')

# Data wrangling

In [ ]:
df_stats_jokic.isnull().sum()

Temporada                     0
Date                          2
ADV                           2
Resultado                     2
MIN                           0
FG                            0
FG%                           0
3PT                           0
3P%                           0
FT                            0
FT%                           0
REB                           0
V                             0
BLK                           0
STL                           0
PF                            0
TO                            0
PTS                           0
Estat. Temporada Regular    654
dtype: int64

In [ ]:
df_stats_jokic.drop(columns=['Estat. Temporada Regular'], inplace=True)

In [ ]:
df_stats_jokic.dropna(axis=0, how='any', inplace=True)

In [ ]:
df_stats_jokic.isnull().sum()

Temporada    0
Date         0
ADV          0
Resultado    0
MIN          0
FG           0
FG%          0
3PT          0
3P%          0
FT           0
FT%          0
REB          0
V            0
BLK          0
STL          0
PF           0
TO           0
PTS          0
dtype: int64

In [ ]:
renamed_columns = {'Date': 'Data',
                   'V': 'Assistencias',
                   'PF': 'Faltas',
                   'TO': 'Tunovers',
                   'MIN': 'Minutos',
                   'REB': 'Rebotes',
                   'BLK': 'Bloqueios',
                   'STL': 'Desarmes',
                   'PTS': 'Pontos'}

df_stats_jokic.rename(columns=renamed_columns, inplace=True)

df_stats_jokic.head()

,Temporada,Data,ADV,Resultado,Minutos,FG,FG%,3PT,3P%,FT,FT%,Rebotes,Assistencias,Bloqueios,Desarmes,Faltas,Tunovers,Pontos
0,2016,Qui 14/4,emPOR,D107-99,35.0,7-10,70.0,1-2,50.0,3-3,100.0,13.0,5.0,0.0,0.0,3.0,2.0,18.0
1,2016,Dom 10/4,xUTAH,D100-84,36.0,7-9,77.8,0-2,0.0,5-6,83.3,11.0,4.0,1.0,2.0,4.0,1.0,19.0
2,2016,Sáb 9/4,xSA,V102-98,32.0,3-9,33.3,0-2,0.0,2-2,100.0,15.0,4.0,1.0,1.0,2.0,0.0,8.0
3,2016,Qua 6/4,xOKC,D124-102,19.0,3-5,60.0,1-3,33.3,0-0,0.0,2.0,2.0,2.0,0.0,2.0,0.0,7.0
4,2016,Dom 3/4,xSAC,D115-106,32.0,2-6,33.3,0-1,0.0,2-2,100.0,13.0,5.0,2.0,1.0,2.0,0.0,6.0


In [ ]:
# Remoção de linhas com texto de nome de meses
df_stats_jokic = df_stats_jokic[(df_stats_jokic['Data']   != 'january')
                                & (df_stats_jokic['Data'] != 'february')
                                & (df_stats_jokic['Data'] != 'march')
                                & (df_stats_jokic['Data'] != 'april')
                                & (df_stats_jokic['Data'] != 'may')
                                & (df_stats_jokic['Data'] != 'june')
                                & (df_stats_jokic['Data'] != 'july')
                                & (df_stats_jokic['Data'] != 'august')
                                & (df_stats_jokic['Data'] != 'september')
                                & (df_stats_jokic['Data'] != 'october')
                                & (df_stats_jokic['Data'] != 'november')
                                & (df_stats_jokic['Data'] != 'december')]

In [ ]:
# Verifica a remoção de linhas com texto de nome de meses
df_stats_jokic[(df_stats_jokic['Data']   == 'january')
               | (df_stats_jokic['Data'] == 'february')
               | (df_stats_jokic['Data'] == 'march')
               | (df_stats_jokic['Data'] == 'april')
               | (df_stats_jokic['Data'] == 'may')
               | (df_stats_jokic['Data'] == 'june')
               | (df_stats_jokic['Data'] == 'july')
               | (df_stats_jokic['Data'] == 'august')
               | (df_stats_jokic['Data'] == 'september')
               | (df_stats_jokic['Data'] == 'october')
               | (df_stats_jokic['Data'] == 'november')
               | (df_stats_jokic['Data'] == 'december')]

,Temporada,Data,ADV,Resultado,Minutos,FG,FG%,3PT,3P%,FT,FT%,Rebotes,Assistencias,Bloqueios,Desarmes,Faltas,Tunovers,Pontos


In [ ]:
# Regex de informações das colunas para novas colunas exclusivas
df_stats_jokic['V_D'] = df_stats_jokic['Resultado'].str.extract(r'(^D|V)')  # ^ garante que está no começo da string

df_stats_jokic['Placar'] = df_stats_jokic['Resultado'].str.extract(r'(\d+-\d+)')

df_stats_jokic['Casa_Fora'] = df_stats_jokic['ADV'].str.extract(r'(em|x)')

df_stats_jokic['Adversario'] = df_stats_jokic['ADV'].str.extract(r'(?:em|x)([A-Z]+)')  # (?:em|x) - non-capturing group that matches 'em' or 'x'

df_stats_jokic['Dia_semana'] = df_stats_jokic['Data'].str.extract(r'([A-Za-z0-9_À-ÿ]+)')

df_stats_jokic['Mes'] = df_stats_jokic['Data'].str.extract(r'\b\d+/\b(\d+)')

<ipython-input-274-54b1c27f33b4>:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-274-54b1c27f33b4>:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-274-54b1c27f33b4>:6: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-274-54b

In [ ]:
df_stats_jokic.tail()

,Temporada,Data,ADV,Resultado,Minutos,FG,FG%,3PT,3P%,FT,...,Desarmes,Faltas,Tunovers,Pontos,V_D,Placar,Casa_Fora,Adversario,Dia_semana,Mes
650,2023,Qui 27/10,xLAL,V110-99,35.0,12-17,70.6,0-4,0.0,7-7,...,4.0,3.0,3.0,31.0,V,110-99,x,LAL,Qui,10
651,2023,Ter 25/10,emPOR,D135-110,27.0,3-4,75.0,0-0,0.0,3-3,...,0.0,5.0,1.0,9.0,D,135-110,em,POR,Ter,10
652,2023,Dom 23/10,xOKC,V122-117,39.0,6-10,60.0,1-1,100.0,6-9,...,1.0,2.0,3.0,19.0,V,122-117,x,OKC,Dom,10
653,2023,Sáb 22/10,emGS,V128-123,34.0,7-13,53.8,1-2,50.0,11-11,...,0.0,4.0,5.0,26.0,V,128-123,em,GS,Sáb,10
654,2023,Qui 20/10,emUTAH,D123-102,33.0,12-17,70.6,1-3,33.3,2-2,...,3.0,4.0,3.0,27.0,D,123-102,em,UTAH,Qui,10


In [ ]:
df_stats_jokic['Casa_Fora'] = df_stats_jokic['Casa_Fora'].map({'em': 'fora','x': 'casa'})

df_stats_jokic['Dia_semana'] = (df_stats_jokic['Dia_semana'].map({'Seg': 0,
                                                                  'Ter': 1,
                                                                  'Qua': 2,
                                                                  'Qui': 3,
                                                                  'Sex': 4,
                                                                  'Sáb': 5,
                                                                  'Dom': 6}))  # Dias da semana no padrão python

df_stats_jokic.drop(columns=['Data', 'ADV', 'Resultado'], inplace=True)

df_stats_jokic.head()

,Temporada,Minutos,FG,FG%,3PT,3P%,FT,FT%,Rebotes,Assistencias,...,Desarmes,Faltas,Tunovers,Pontos,V_D,Placar,Casa_Fora,Adversario,Dia_semana,Mes
0,2016,35.0,7-10,70.0,1-2,50.0,3-3,100.0,13.0,5.0,...,0.0,3.0,2.0,18.0,D,107-99,fora,POR,3,4
1,2016,36.0,7-9,77.8,0-2,0.0,5-6,83.3,11.0,4.0,...,2.0,4.0,1.0,19.0,D,100-84,casa,UTAH,6,4
2,2016,32.0,3-9,33.3,0-2,0.0,2-2,100.0,15.0,4.0,...,1.0,2.0,0.0,8.0,V,102-98,casa,SA,5,4
3,2016,19.0,3-5,60.0,1-3,33.3,0-0,0.0,2.0,2.0,...,0.0,2.0,0.0,7.0,D,124-102,casa,OKC,2,4
4,2016,32.0,2-6,33.3,0-1,0.0,2-2,100.0,13.0,5.0,...,1.0,2.0,0.0,6.0,D,115-106,casa,SAC,6,4


## Salva dataset ajustado

In [ ]:
# df_stats_jokic.to_csv('/content/drive/MyDrive/NBA Analysis/nba_stats_jokic_ajustado.csv',
#                       encoding='utf-8',
#                       sep=',',
#                       index=False)

# Dataset from csv

In [ ]:
df_stats_jokic = pd.read_csv('/content/drive/MyDrive/NBA Analysis/nba_stats_jokic_ajustado.csv',
                             encoding='utf-8',
                             sep=',')
df_stats_jokic.head()

,Temporada,Minutos,FG,FG%,3PT,3P%,FT,FT%,Rebotes,Assistencias,...,Desarmes,Faltas,Tunovers,Pontos,V_D,Placar,Casa_Fora,Adversario,Dia_semana,Mes
0,2016,35.0,7-10,70.0,1-2,50.0,3-3,100.0,13.0,5.0,...,0.0,3.0,2.0,18.0,D,107-99,fora,POR,3,4
1,2016,36.0,7-9,77.8,0-2,0.0,5-6,83.3,11.0,4.0,...,2.0,4.0,1.0,19.0,D,100-84,casa,UTAH,6,4
2,2016,32.0,3-9,33.3,0-2,0.0,2-2,100.0,15.0,4.0,...,1.0,2.0,0.0,8.0,V,102-98,casa,SA,5,4
3,2016,19.0,3-5,60.0,1-3,33.3,0-0,0.0,2.0,2.0,...,0.0,2.0,0.0,7.0,D,124-102,casa,OKC,2,4
4,2016,32.0,2-6,33.3,0-1,0.0,2-2,100.0,13.0,5.0,...,1.0,2.0,0.0,6.0,D,115-106,casa,SAC,6,4


## Correlação

In [ ]:
correlacao = df_stats_jokic[['Temporada',
                             'Minutos',
                             'FG%',
                             '3P%',
                             'FT%',
                             'Rebotes',
                             'Assistencias',
                             'Bloqueios',
                             'Desarmes',
                             'Faltas',
                             'Tunovers',
                             'Pontos',
                             'Mes']].corr().round(2)

fig = px.imshow(correlacao, text_auto=True, aspect='auto')

fig.update_layout(margin=dict(t=50,
                              r=50,
                              b=50,
                              l=50),
                  showlegend=False,
                  width=1200,
                  height=1200)

fig.show()